In [2]:
import sys
import os
import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
import matplotlib
import warnings

In [53]:
# path1 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/Reprocessed_20241003_withMoNA/Area_1_2024_10_03_09_05_38_copy.csv'
#path3 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/Chemical List PLASTICMAP-2024-10-03.csv'
path3 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/CCD-Batch-Search_2024-10-21_07_17_40.csv'
path1 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Area_1_2024_10_22_17_50_44_neg.csv'
path2 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/sample_meta_neg.csv'
path5 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/sample_meta_pos.csv'
path4 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Area_1_2024_10_22_17_13_52_pos.csv'
path6 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/QCtest/QCtest/Area_1_2024_10_28_13_47_02.csv'

In [4]:
# Load the targeted CSV file
chemical_df = pd.read_csv(path3, encoding='ISO-8859-1')
print(chemical_df.shape)
# Retain rows with available value in SMILES column
chemical_df = chemical_df[chemical_df['MS_READY_SMILES'].notna()]
# chemical_df = chemical_df[chemical_df['SMILES'].notna()]
print(chemical_df.shape)

C:\Users\yangj\AppData\Local\Temp\ipykernel_49356\1155436258.py:2: DtypeWarning: Columns (19,20,59) have mixed types. Specify dtype option on import or set low_memory=False.
  chemical_df = pd.read_csv(path3, encoding='ISO-8859-1')


(10547, 794)
(6375, 794)


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdmolops import RemoveHs


# Function to sanitize the SMILES
def sanitize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None, None

        # Remove fragments after removing salts
        frags = Chem.GetMolFrags(mol, asMols=True, sanitizeFrags=False)
        mol = max(frags, default=None, key=lambda m: m.GetNumAtoms())
        if mol is None:
            return None, None

        # Attempt to sanitize molecule
        Chem.SanitizeMol(mol)

        # Standardize and calculate monoisotopic mass
        standardized_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
        monoisotopic_mass = Descriptors.ExactMolWt(mol)
        
        return standardized_smiles, monoisotopic_mass
    except Exception as e:
        print(f"Error sanitizing SMILES {smiles}: {e}")
        return None, None

SMILES_ready =[]
Monoisotopic_Mass_ready =[]
for smiles in chemical_df['SMILES']:
    standardized_smiles, monoisotopic_mass = sanitize_smiles(smiles)
    if standardized_smiles:
        SMILES_ready.append(standardized_smiles)
        Monoisotopic_Mass_ready.append(monoisotopic_mass)
    else:
        SMILES_ready.append('NA')
        Monoisotopic_Mass_ready.append('NA')
chemical_df['SMILES_ready'] = SMILES_ready
chemical_df['Monoisotopic_Mass_ready'] = Monoisotopic_Mass_ready

In [6]:
#clean up targeted molecule list, retain organic molecules only
import pandas as pd
from rdkit import Chem

# # Retain rows with available value in SMILES column
chemical_df = chemical_df[chemical_df['SMILES_ready'].notna()]

# Convert 'MONOISOTOPIC MASS' column to numeric, forcing errors to NaN
chemical_df['Monoisotopic_Mass_ready'] = pd.to_numeric(chemical_df['Monoisotopic_Mass_ready'], errors='coerce')

# Drop rows with NaN in 'MONOISOTOPIC MASS'
chemical_df = chemical_df[chemical_df['Monoisotopic_Mass_ready'].notna()]

# Function to strictly determine if a molecule is organic
def is_organic(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return False  # invalid SMILES
    organic_elements = {'C', 'H', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I'}  # Common organic elements
    inorganic_elements = {'Li', 'Be', 'Na', 'Mg', 'Al', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Th', 'U'}
    contains_carbon = False
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        if symbol == 'C':
            contains_carbon = True
        if symbol in inorganic_elements:
            return False  # Contains metal or other inorganic elements
        if symbol not in organic_elements:
            return False  # Contains uncommon elements for organic molecules
    return contains_carbon

# Step 1: Filter rows with available MW (MONOISOTOPIC MASS) between 100 and 1000 and classified as organic
filtered_df = []
for index, row in chemical_df.iterrows():
    smiles = row['SMILES_ready']  # Assuming the SMILES column is named 'SMILES'
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mw = row['Monoisotopic_Mass_ready']  # Assuming the column is named 'MONOISOTOPIC MASS'
        if 100 <= mw <= 1000:
            if is_organic(smiles):
                filtered_df.append(row)

# Create a new DataFrame from the filtered rows
filtered_df = pd.DataFrame(filtered_df)

# Save the filtered DataFrame to a new CSV file
# filtered_df.to_csv('D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/Filtered_Chemical_List.csv', index=False)

print("Filtering complete. The filtered list has been saved as 'Filtered_Chemical_List.csv'.")

Filtering complete. The filtered list has been saved as 'Filtered_Chemical_List.csv'.


In [ ]:
##the above code is for filtering the chemical list: by selecting the organic compounds from the plasticmap chemical list

In [ ]:
filtered_df.drop_duplicates(subset= 'SMILES_ready', inplace=True)
filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv', index=False)
Export_Transformation = filtered_df[['SMILES_ready']]
# Export_Transformation.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic_forEPA_CTS.txt', index=False)

In [54]:
#the following code is used for fitered plastic chemicals based on ms1 search on the first 100 batch sample data.
#and use the chemical list to generate the insilico spectrum database.
#update the search with the filtered ms1 peak area file and return the matched results for the plastic map chemcial list

filtered_df = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')

from tqdm import tqdm
# Load LCMS processed data with peak areas (MSDIAL result file)
msdial_df = pd.read_csv(path4, encoding='ISO-8859-1') #
print(msdial_df.shape) #(13319, 52)

# Retain QC and Blank samples only
metafile = pd.read_csv(path5, encoding='ISO-8859-1')
qc_blank_samples = metafile[metafile['File type'].isin(['QC', 'Blank'])]['MS/MS spectrum'].tolist()

# Filter MSDIAL result file for QC and Blank samples
# qc_blank_msdial_df = msdial_df[['Average Rt(min)', 'Average Mz'] + [col for col in msdial_df.columns if col in qc_blank_samples]]
qc_blank_msdial_df = msdial_df[['Average Rt(min)', 'Average Mz']]

# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 10
filtered_df['Hits'] = 0
matched_rows = []

for _, chem_row in tqdm(filtered_df.iterrows(), total=filtered_df.shape[0], desc="Matching compounds"):
    mw = chem_row['MONOISOTOPIC_MASS']
    matches = qc_blank_msdial_df[(qc_blank_msdial_df['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        filtered_df.at[chem_row.name, 'Hits'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'PREFERRED NAME': chem_row.get('PREFERRED NAME', None), 'DTXSID': chem_row.get('DTXSID', None)})


# Create a new DataFrame for matched MSDIAL result rows
matched_msdial_df = pd.DataFrame(matched_rows)
filtered_df = filtered_df[filtered_df['Hits']>0]
filtered_df.shape #(1244, 797)

# Save the updated filtered DataFrame and matched MSDIAL DataFrame to CSV files
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Matched_MSDIAL_Results.csv', index=False)
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Matched_MSDIAL_Results.csv', index=False)

# Print the number of chemicals with hits from the MSDIAL result file
num_chemicals_with_hits = filtered_df[filtered_df['Hits'] > 0].shape[0]
print(f"Number of chemicals in the filtered_df with hits from the MSDIAL result file: {num_chemicals_with_hits}")
print("Matching and file generation complete.")

C:\Users\yangj\AppData\Local\Temp\ipykernel_31932\1078982296.py:5: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')


(13319, 52)


Matching compounds: 100%|██████████| 5144/5144 [00:36<00:00, 141.31it/s]

Number of chemicals in the filtered_df with hits from the MSDIAL result file: 620
Matching and file generation complete.


In [ ]:
#####
#generate the files for CFM-ID spectra prediciton
CFMID_ready_df = filtered_df[['MS_READY_SMILES']].copy()
# Split each value by ',' and retain the longest string
CFMID_ready_df['MS_READY_SMILES'] = CFMID_ready_df['MS_READY_SMILES'].apply(
    lambda x: max(x.split(','), key=len) if isinstance(x, str) else x
)

CFMID_ready_df['ID'] = ['Molecule' + str(i + 1) for i in range(len(CFMID_ready_df))]
CFMID_ready_df = CFMID_ready_df[['ID','MS_READY_SMILES']]
# CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_CFMID_MSready.txt', sep='\t', index=False, header=False)
CFMID_ready_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_CFMID_MSready.txt', sep='\t', index=False, header=False)

In [15]:
import pandas as pd
filtered_df = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
Export_Transformation = filtered_df[['SMILES_ready']]
Export_Transformation =Export_Transformation.rename(columns={'SMILES_ready':'SMILES'})
Export1 = Export_Transformation.iloc[0:2000,]
Export2 = Export_Transformation.iloc[2000:4000,]
Export3 = Export_Transformation.iloc[4000:5144,]
Export1.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch1_biotran.csv', index = False)
Export2.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch2_biotran.csv', index = False)
Export3.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch3_biotran.csv', index = False)

C:\Users\yangj\AppData\Local\Temp\ipykernel_24948\1615265377.py:2: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')


In [ ]:
Export1.shape[0] +Export2.shape[0] +Export3.shape[0] #5144
filtered_df.shape #5144

5144

In [6]:
##clean up and combined the biotransformation product
##combined with the precursor molecules for the biotransformation product.
##combined into a big list.
import pandas as pd
import os

original_suspectlst = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
filepath = 'D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/'
file1 = 'bath1_ouput.csv'
file2 = 'bath2_ouput.csv'
file3 = 'bath3_ouput.csv'
transform1 = pd.read_csv(os.path.join(filepath,file1))
transform2 = pd.read_csv(os.path.join(filepath,file2))
transform3 = pd.read_csv(os.path.join(filepath,file3))

transform_combined = pd.concat([transform1,transform2,transform3],axis=0)
transform_combined.shape

C:\Users\yangj\AppData\Local\Temp\ipykernel_31932\2704480760.py:7: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  original_suspectlst = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')


(26838, 23)

In [ ]:
##filter duplicated row with same SMILES, filter rows with mono isotope mass around 100-1000
filtered_transform = transform_combined[(transform_combined['Major Isotope Mass'] >=100)&(transform_combined['Major Isotope Mass'] <=1000)]
unique_product = filtered_transform['SMILES'].value_counts()[filtered_transform['SMILES'].value_counts()==1].index
print(unique_product.shape) #25871,23
unique_precursor = filtered_transform['Precursor SMILES'].value_counts()[filtered_transform['Precursor SMILES'].value_counts()==1].index
print(unique_precursor.shape) 
unique_transform = filtered_transform[filtered_transform['SMILES'].isin(unique_product)] #23205,466
unique_transform.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/unique_biotransformation_product_combined_mass100_1000.csv", index=False )

(25871, 23)


In [25]:
unique_precursor_orig = original_suspectlst['SMILES_ready'].value_counts()[original_suspectlst['SMILES_ready'].value_counts()==1].index
print(unique_precursor_orig.shape)

(5144,)


In [ ]:
unique_transform.columns
unique_transform.head()

,InChI,InChIKey,SMILES,Synonyms,PUBCHEM_CID,Molecular formula,Major Isotope Mass,ALogP,Lipinski_Violations,Insecticide_Likeness_Violations,...,Reaction ID,Enzyme(s),Biosystem,Precursor ID,Precursor SMILES,Precursor InChI,Precursor InChIKey,Precursor ALogP,Precursor Major Isotope Mass,Hits
0,InChI=1S/C13H22O3/c1-9(14)12(15)16-11-7-5-10(6...,FPCXHNBSKIAECM-UHFFFAOYSA-N,C=C(C(=O)OC1CCC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,3.1799,0,0,...,BTMR1055,CYP1A2\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,4
1,InChI=1S/C13H22O3/c1-5-12(15)16-11-7-6-9(8-10(...,SHFANGIQSWVUTM-UHFFFAOYSA-N,C=CC(=O)OC1C(CC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,2.7761,0,0,...,BTMR1068,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,4
2,InChI=1S/C13H22O3/c1-5-12(15)16-9-6-7-10(11(14...,XKHJMOQTQPIDEI-UHFFFAOYSA-N,C=CC(=O)OC1CC(C(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,2.4498,0,0,...,BTMR1068,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,1
3,"InChI=1S/C13H22O3/c1-13(2,3)9-4-6-10(7-5-9)16-...",UTLAETFBDLVAFS-UHFFFAOYSA-N,C1C(C(=O)OC2CCC(C(C)(C)C)CC2)O1,NaN,NaN,C13H22O3,226.156895,2.8426,0,0,...,BTMR0109,CYP1A2\nCYP2B6\nCYP2B6\nCYP2C8\nCYP2C9\nCYP2C1...,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,1
4,InChI=1S/C13H22O3/c1-4-12(15)16-11-7-5-10(6-8-...,YTMFLOJRIFWYDD-UHFFFAOYSA-N,C=CC(=O)OC1CCC(C(C)(C)CO)CC1,NaN,NaN,C13H22O3,226.156895,2.6523,0,0,...,BTMR1061,CYP1A2\nCYP2A6\nCYP2B6\nCYP2C8\nCYP2C9\nCYP2C1...,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,1


In [ ]:
#do ms1 search and matching for the pos peak area for the biotransformation product
#import peak area for positive after batch correction
#pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc.csv")
    #ms1 search based on the Average Rt and Average Mz
from tqdm import tqdm
targetmzdat = unique_transform
peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]

# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 20
targetmzdat['Hits_neg'] = 0
matched_rows = []

for _, chem_row in tqdm(targetmzdat.iterrows(), total=targetmzdat.shape[0], desc="Matching compounds"):
    mw = chem_row['Major Isotope Mass']
    matches = peak_area_inquiry[(peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        targetmzdat.at[chem_row.name, 'Hits_neg'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'PUBCHEM_CID': chem_row.get('PUBCHEM_CID', None), 'Precursor SMILES': chem_row.get('Precursor SMILES', None), 
                                 'Reaction': chem_row.get('Reaction', None)})


# Create a new DataFrame for matched MSDIAL result rows
matched_peak_area_df = pd.DataFrame(matched_rows)
targetmzdat = targetmzdat[targetmzdat['Hits_neg']>0]
# targetmzdat.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/filtered_biotransform_product_list_pos.csv") #(13098, 24)
targetmzdat.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/filtered_biotransform_product_list_neg.csv") #(11018, 25)
targetmzdat.shape

# Save the updated filtered DataFrame and matched MSDIAL DataFrame to CSV files
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Matched_MSDIAL_Results.csv', index=False)
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Matched_MSDIAL_Results.csv', index=False)

# Print the number of chemicals with hits from the MSDIAL result file
# num_chemicals_with_hits = filtered_df[filtered_df['Hits'] > 0].shape[0]


C:\Users\yangj\AppData\Local\Temp\ipykernel_31932\2869617335.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targetmzdat['Hits_neg'] = 0
Matching compounds: 100%|██████████| 23205/23205 [02:15<00:00, 171.34it/s]


(11018, 25)

In [51]:
targetmzdat.head()

,InChI,InChIKey,SMILES,Synonyms,PUBCHEM_CID,Molecular formula,Major Isotope Mass,ALogP,Lipinski_Violations,Insecticide_Likeness_Violations,...,Enzyme(s),Biosystem,Precursor ID,Precursor SMILES,Precursor InChI,Precursor InChIKey,Precursor ALogP,Precursor Major Isotope Mass,Hits,Hits_neg
0,InChI=1S/C13H22O3/c1-9(14)12(15)16-11-7-5-10(6...,FPCXHNBSKIAECM-UHFFFAOYSA-N,C=C(C(=O)OC1CCC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,3.1799,0,0,...,CYP1A2\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,4,1
1,InChI=1S/C13H22O3/c1-5-12(15)16-11-7-6-9(8-10(...,SHFANGIQSWVUTM-UHFFFAOYSA-N,C=CC(=O)OC1C(CC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,2.7761,0,0,...,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,4,1
6,InChI=1S/C24H44O5/c1-5-9-11-18(7-3)16-28-23(26...,RQQVQEYHBZVRCH-UHFFFAOYSA-N,CCCCC(CC)COC(=O)C1C(CC(C(=O)OCC(CC)CCCC)CC1)O,NaN,NaN,C24H44O5,412.318874,6.4891,1,1,...,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,CCCCC(CC)COC(=O)C1CCC(C(=O)OCC(CC)CCCC)CC1,InChI=1S/C24H44O4/c1-5-9-11-19(7-3)17-27-23(25...,HOQGHOMLEVKTBY-UHFFFAOYSA-N,NaN,396.3239,11,1
10,InChI=1S/C24H44O5/c1-4-7-9-19(6-3)17-28-23(26)...,KRGKATBUQOGPTC-UHFFFAOYSA-N,CCCCC(CC)COC(=O)C1CCC(C(=O)OCC(CCO)CCCC)CC1,NaN,NaN,C24H44O5,412.318874,6.3653,1,1,...,CYP1A2\nCYP2A6\nCYP2B6\nCYP2C8\nCYP2C9\nCYP2C1...,HUMAN,NaN,CCCCC(CC)COC(=O)C1CCC(C(=O)OCC(CC)CCCC)CC1,InChI=1S/C24H44O4/c1-5-9-11-19(7-3)17-27-23(25...,HOQGHOMLEVKTBY-UHFFFAOYSA-N,NaN,396.3239,4,5
11,InChI=1S/C24H42O4/c1-5-9-11-19(7-3)17-27-23(25...,UIKOVSPOHUIQOO-UHFFFAOYSA-N,C=CCCC(CC)COC(=O)C1CCC(C(=O)OCC(CC)CCCC)CC1,NaN,NaN,C24H42O4,394.308310,7.2003,1,1,...,CYP1A2\nCYP2A6\nCYP2C9\nCYP2D6\nCYP3A4,HUMAN,NaN,CCCCC(CC)COC(=O)C1CCC(C(=O)OCC(CC)CCCC)CC1,InChI=1S/C24H44O4/c1-5-9-11-19(7-3)17-27-23(25...,HOQGHOMLEVKTBY-UHFFFAOYSA-N,NaN,396.3239,4,5


In [ ]:
##detection frequency of thoes features with hits, include the biotransformation product and perform ms1 search again. in R
##calculate standard deviations and covariance of those features among 200 serum samples
##generate the final inclusion list for ms/ms acquisition
##with the ms2, perform molecular networking to support the plastic and their biotrnasformation products
